In [ ]:
import os

import numpy
from matplotlib import pyplot as plt

from fit_diagnostics import abs_residuals, norm_residuals
from plot_utils import cplot, plot_red_vis
from red_likelihood import condenseMap, degVis, doRelCal, doRelCalRP, doOptCal, doDegVisVis, \
flt_ant_pos, group_data, gVis, makeEArray, norm_rel_sols, red_ant_sep, relabelAnts, rotate_phase
from red_utils import find_flag_file, find_nearest, find_zen_file, get_bad_ants, \
match_lst, split_rel_results

In [ ]:
numpy.set_printoptions(threshold=500)

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

In [ ]:
JD = 2458098.43869
pol = 'ee'
freq_channel = 605
time_integration = 0
distribution = 'cauchy' # fitting distribution for neg log-likelihood minimizations
rel_cal_coords = 'polar' # parameter coordinate system
bounded_rel_cal = False # bound gain and visibility amplitudes in relative calibration
RP_results = True # use reduced parameter relative calibration method
rot_phase = False # rotate phases of relative calibration gains with negative amplitudes 

In [ ]:
zen_fn = find_zen_file(JD)
bad_ants = get_bad_ants(zen_fn)
flags_fn = find_flag_file(JD, 'first') # import flags from firstcal

In [ ]:
hdraw, RedG, cMData = group_data(zen_fn, pol, freq_channel, None, bad_ants, flags_fn)
cData = cMData.filled() # filled with nans for flags
flags = cMData.mask

# mitigating for multiple freqs - only chooses first one
if cData.shape[0] > 1:
    cData = cData[0, ...]
    flags = flags[0, ...]
    print('Frequency channel {} selected for notebook analysis\n'.format(freq_channel[0]))
cData = numpy.squeeze(cData)
flags = numpy.squeeze(flags)

if all(numpy.isnan(cData[time_integration, :])):
    print('All visibilities for channel {} and time integration {} are flagged '\
          '- choose different values'.format(freq_channel, time_integration))

ants = numpy.unique(RedG[:, 1:])
no_ants = ants.size
no_unq_bls = numpy.unique(RedG[:, 0]).size
cRedG = relabelAnts(RedG)

In [ ]:
plot_red_vis(cData, RedG, vis_type='amp')

# Redundant calibration

Fundamentally, the problem of calibration boils down to the measurement equation:

$$ V_{ij}^{\text{obs}} (\nu) = g_i (\nu) g_j^* (\nu) V_{ij}^{\text{true}}(\nu) + n_{ij} (\nu) $$

where the observed visibility $V_{ij}^{\text{obs}}$ between antennas $i$ and $j$ at a given time and frequency is related to the true underlying visibility $V_{ij}^{\text{true}}$ by a pair of complex and frequency-dependent gain factors, $g_i$ and $g_j$, if we assume per-antenna gains, along with uncorrelated Gaussian random noise $n_{ij}$. The ultimate aim of calibration is to solve for these gains and true visibilities. 

An array with regularly spaced antennas has few unique baselines, hence there are many redundant visibilities with precisely the same baseline separation between them that are sensitive to the very same modes on the sky. In redundant calibration, we impose the prior that the true sky visibilities $V_{ij}^{\text{true}}$ from redundant baselines are equal. We therefore have a system of equations for all antenna pairs $i$ and $j$, given by

$$ V_{ij}^{\text{obs}} (\nu) = g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) + n_{ij} (\nu) $$

where $U_{\alpha}(\nu) = V(\mathbf{r}_i-\mathbf{r}_j)$, the visibility for the baseline vector $\mathbf{b}_{ij} = \mathbf{r}_i-\mathbf{r}_j$, corresponds to a redundant baseline set that we index by $\alpha$.

In highly redundant arrays, like \gls{hera}, there are many more observations than there are unique baselines. For the full HERA array, there are 331 elements in the hexagonal core, corresponding to $N_{\mathrm{bl}} = 331(331-1)/2 = 54,615$ baselines. The hexagonal core only has 630 unique baseline separations, this means that we have a non-linear system of $54,615$ equations to determine the 630 unique true visibilities and the 331 complex gains; the system is vastly overdetermined. 

## Relative calibration

Relative calibration is the process whereby true sky visibilities from redundant baselines are set to be equal (i.e. $V_{ij}^{\text{true}} = U_{\alpha}(\nu)$). From this prior, an MLE for the gains and true visibilities can be constructed by assuming a distribution for the observed visibility noise. However, during this process, degeneracies arise, which must be constrained with absolute calibration. We call relative calibration the part that solves up to the degenerate parameters, and absolute calibration the part that constrains them. Redundant calibration consists of both of these components together.

### Gaussian distribution

By imposing that the true sky visibilities from redundant baselines are equal, relative redundant calibration solves for the true sky visibilities for each redundant baseline set $V_{i-j}^{\text{sol}}$, alongside the gains of the individual antennas.

Assuming Gaussian uncorrelated noise with variance $\sigma_{ij}^2$, the \gls{mle} in solving for the gains and true visibilities produces the following likelihood function:

$$ 	\mathcal{L}^G_{\mathrm{rel}} (\{g_i(\nu)\}, \{U_{\alpha}(\nu)\} | \{V_{ij}^{\text{obs}}(\nu)\} \propto \prod_{\nu} \prod_{\alpha} \prod_{\{i,j\}_{\alpha}} \exp\left( -\frac{1}{2} \frac{ \left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^{*} (\nu) U_{\alpha}(\nu) \right|^2}{\sigma_{ij}^2(\nu)} \right) $$

where $\{i,j\}_{\alpha}$ are sets of antennas that belong to each redundant baseline type $\alpha$. Maximizing this function is equivalent to minimizing:

$$ \chi_{\mathrm{rel}}^{2} (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|^2 }{\sigma_{ij}^2(\nu)} $$

by varying $g_k(\nu)$ and $U_{\alpha}(\nu)$ for each frequency $\nu$.

This non-linear least-squares optimization can be done independently between frequencies and time (although adjacent solutions in time and frequencies are expected to be very similar). Solving the above $\chi_{\mathrm{rel}}^{2}$ has been the main focus of redundant calibration methods, with many opting to linearize the redundant measurement equation for computational ease. In this work, we follow through with the full $\chi_{\mathrm{rel}}^{2}$ evaluation, as well as assuming other model distributions in the MLE, with the belief that such computations can be greatly accelerated with novel software and GPUs.

### Cauchy distribution

Empirically, it is found that the noise from visibility observations from a redundant set may not be Gaussian, due to non-redundancies from instrumental effects and the presence of outliers (from e.g. RFI), and that the visibilities may follow a distribution with fatter tails. We wish to be insensitive to outliers, so we need to employ robust statistics to adequately deal with such measurements.

It is clear that outliers will affect any Gaussian or mean-centred fitting, therefore resulting in inaccurate gain and true sky visibility solutions. The Cauchy distribution, however, is median-centred (its mean is undefined); it is given by

$$ f(x; x_0, \gamma) = \frac{1}{\pi \gamma \left[ 1 + \left( \frac{x - x_0}{\gamma} \right)^2 \right] }$$

where $x_0$ is the location parameter (the median) and $\gamma$ is the scale parameter, which specifies the HWHM. These are both robust measures of central tendency and statistical dispersion, respectively, that are not unduly affected by outliers..

We no longer assume that the noise in the measurement equation is Gaussian, and instead, assume it is Cauchy distributed when solving for the relative redundant calibration parameters.

Working in a maximum likelihood framework, the likelihood when solving for the measurement equation for redundant baseline sets is given by

$$ \mathcal{L}^C_{\mathrm{rel}} (\{g_i(\nu)\}, \{U_{\alpha}(\nu)\} | \{V_{ij}^{\text{obs}}(\nu)\} = \\ \prod_{\nu} \prod_{\alpha} \prod_{\{i,j\}_{\alpha}} \frac{1}{\pi \gamma_{\alpha} (\nu)} \left[ 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|}{\gamma_{\alpha} (\nu)} \right)^2 \right]^{-1} $$

The negative log-likelihood is therefore given by

$$ -\ln(\mathcal{L}^C_{\mathrm{rel}}) (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha} (\nu)) + \ln \left( 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|}{\gamma_{\alpha}(\nu)} \right)^2 \right) $$

where we have dropped the sum over frequency, as we can solve independently for each frequency.

In [ ]:
obs_vis = cData[time_integration, :]
res_rel = doRelCal(cRedG, obs_vis, no_unq_bls, no_ants, distribution=distribution, \
                   coords=rel_cal_coords, bounded=bounded_rel_cal, norm_gains=True)
res_relx = numpy.array(res_rel['x'])
if rel_cal_coords == 'polar' and rot_phase and (res_relx[-2*no_ants::2] < 0).any():
    # adjustement if negative amplitude solutions are found, where the absolute values
    # of amplitudes are taken, and phases of affected antennas are rotated by +pi
    print('Rotating gain and visibility phases to have positive amplitudes.')
    res_relx = rotate_phase(res_relx, no_unq_bls, norm_gains=True)
    
res_rel_vis, res_rel_gains = split_rel_results(res_relx, no_unq_bls, \
                                               coords=rel_cal_coords)

In [ ]:
# gain amplitudes
if rel_cal_coords == 'polar':
    print(res_relx[-2*no_ants::2])
if rel_cal_coords == 'cartesian':
    print(numpy.abs(res_rel_gains))

In [ ]:
# gain phases
if rel_cal_coords == 'polar':
    print(res_relx[-2*no_ants+1::2])
if rel_cal_coords == 'cartesian':
    print(numpy.angle(res_rel_gains))

In [ ]:
# try JAX minimization
jres_rel = doRelCal(cRedG, obs_vis, no_unq_bls, no_ants, distribution=distribution, \
                    coords=rel_cal_coords, jax_minimizer=True)

In [ ]:
# Try with alternative calibration method whereby the number of parameters in the
# minimization is reduced such that the average gain amplitude and phase are constrained
res_relRP, _ = doRelCalRP(cRedG, obs_vis, no_unq_bls, no_ants, distribution=distribution, \
                          constr_phase=True, amp_constr='prod', bounded=True)
res_relRPx = numpy.array(res_relRP['x'])
if rot_phase:
    res_relRPx = rotate_phase(res_relRPx, no_unq_bls)
RP_vis, RP_gains = split_rel_results(res_relRPx, no_unq_bls, coords='polar')
RP_mean = numpy.mean(numpy.abs(RP_gains))
print('Gains - average amp: {}, product of amps: {}, average phase: {}'.format(RP_mean, \
      numpy.prod(numpy.abs(RP_gains)), numpy.mean(numpy.angle(RP_gains))))

In [ ]:
# jres_relRP, _ = doRelCalRP(cRedG, obs_vis, no_unq_bls, no_ants, distribution=distribution, \
#                            constr_phase=True, bounded=False, amp_constr='prod', jax_minimizer=False)

In [ ]:
# Gain amplitudes for the reduced parameter method
res_relRPx[::2][-no_ants:]

In [ ]:
if RP_results:
    res_rel = res_relRP
    res_relx = res_relRPx
    res_rel_gains = RP_gains
    res_rel_vis = RP_vis

In [ ]:
# Residuals for relative redundant step
pred_rel_vis = gVis(res_rel_vis, cRedG, res_rel_gains)
rel_residuals = obs_vis - pred_rel_vis
cplot(rel_residuals, xlabel='Baseline', ylabel='Residual')

In [ ]:
# Relative residuals normalized by amplitude
norm_rel_residuals = norm_residuals(obs_vis, pred_rel_vis)
cplot(norm_rel_residuals, xlabel='Baseline', ylabel='Normalized residual')
print('Median absolute normalized residual - Real: {}, Imag: {}'\
      .format(*abs_residuals(norm_rel_residuals)))

## Optimal calibration (solving for degeneracies)

Relative calibration yields degenerate solutions, which can be parameterized as four terms per frequency:
 - Overall amplitude $A(\nu)$
 - Overall phase $\Delta(\nu)$
 - Phase gradient components $\Delta_x(\nu)$ and $\Delta_y(\nu)$
 
since the below transformations of these degenerate parameters leave $-\ln(\mathcal{L})$ (for both the Gaussian and Cauchy distributions) unchanged:
 - $g_i \rightarrow A g_i$ accompanied by $U_{\alpha} \rightarrow A^{-2} U_{\alpha}$
 - $g_k = |g_k|e^{i\phi_k} \rightarrow |g_k|e^{i(\phi_k + \Delta)}$, s.t. $g_k g_l^{*} = |g_k| |g_l| e^{i(\phi_k - \phi_l)} \rightarrow |g_k| |g_l| e^{i(\phi_k + \Delta - \phi_l - \Delta)} = g_k g_l^{*}$
 - $g_k = |g_k|e^{i\phi_k} \rightarrow |g_k|e^{i(\phi_k + \Delta_x x_k + \Delta_y y_k)}$ accompanied by $U_{\alpha} = |U_{\alpha}| e^{i\phi_{\alpha}} \rightarrow |U_{\alpha}| e^{i(\phi_{\alpha} - \Delta_x x_{\alpha} - \Delta_y y_{\alpha})}$
 
where in the last line, the array is assumed to be co-planar, and $(x_k, y_k)$ are the positional coordinates of antenna $k$, and $(x_{\alpha}, y_{\alpha})$ are the separations of the antennas that form baselines in redundant set $\alpha$.

These degeneracies must be constrained - this is the absolute part of redundant calibration. The degenerate parameters can be calculated from a sky model in an absolute calibration step. Alternatively, the degenerate parameters can be solved for by using optimal absolute calibration, which calculates the degenerate parameters directly from the $\chi^2$ or $-\ln(\mathcal{L})$ by applying a few conditions. This latter method, however, still ultimately needs to reference the sky to set the flux scale and pointing centre. These two absolute calibration methods are not mathematically equivalent, but yield consistent results. We describe optimal absolute calibration below, for both Gaussian and Cauchy distributed visibilities.

We define a set of parameters $h_i$ to be the gains that obey the following constraints:

$$
\begin{align}
    \frac{1}{N} \sum_i^N |h_i| = 1 \quad \rightarrow \quad & \text{mean gain amplitude of 1} \\
	\frac{1}{N} \sum_i^N \mathrm{Arg} (h_i) = 0 \quad \rightarrow \quad & \text{mean gain phase of 0} \\
    \sum_i^N x_i \mathrm{Arg} (h_i) = 0 \quad \rightarrow \quad & \text{phase gradient of 0 in }x \\
	\sum_i^N y_i \mathrm{Arg} (h_i) = 0 \quad \rightarrow \quad & \text{phase gradient of 0 in }y
\end{align}
$$

such that the antenna gains can be written as

$$ 	g_i (\nu) = A(\nu) e^{i \left[ \Delta (\nu) + \Delta_{x} (\nu) x_{i} + \Delta_{y} (\nu) y_{i} \right]} h_i (\nu) $$

where $(x_i, y_i)$ is the position of antenna $i$, and all degenerate dependencies have been removed out of $h_i$.

Non-degenerate formulations of are therefore given by 

 - Gaussian distribution:
 
$$ \chi_{\text{opt}}^2 (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left| V_{ij}^{\text{obs}} (\nu) - h_i (\nu) h_j^{*} (\nu) W_{\alpha} (\nu) \right|^2 }{\sigma_{ij}^2(\nu)} $$

 - Cauchy distribution:

$$ -\ln(\mathcal{L}^C_{\mathrm{opt}}) (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha} (\nu)) + \ln \left( 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - h_i (\nu) h_j^{*} (\nu) W_{\alpha} (\nu) \right|}{\gamma_{\alpha}(\nu)} \right)^2 \right) $$

where

$$ W_{\alpha} (\nu) = A^2(\nu) e^{i \left[ \Delta_{x} (\nu) x_{\alpha} + \Delta_{y} (\nu) y_{\alpha} \right]} U_{\alpha} $$

and $(x_{\alpha}, y_{\alpha})$ are the baseline coordinates of redundant set $\alpha$.
 
The overall phase is also degenerate, and is set by requiring that the phase of the gain of a reference antenna is null:

$$ \mathrm{Arg} (h_\text{ref}) = 0 $$

In [ ]:
ref_ant = 85 # to set the overall phase
ref_ant_idx = condenseMap(ants)[ref_ant]
ant_pos = flt_ant_pos(hdraw.antpos, ants)
ant_sep = red_ant_sep(RedG, hdraw.antpos)

res_opt = doOptCal(cRedG, obs_vis, no_ants, ant_pos, ant_sep, res_rel_vis, \
                   distribution=distribution, ref_ant_idx=ref_ant_idx)

In [ ]:
# degenerate parameters
res_opt['x'][-4:]

In [ ]:
new_gain_params, new_deg_params = numpy.split(res_opt['x'], [no_ants*2,])
new_amps = new_gain_params[:no_ants*2:2]
new_phases = new_gain_params[1:no_ants*2:2]
new_gains = makeEArray(new_gain_params)

In [ ]:
print('Degenerate parameters: {}'.format(str(new_deg_params)[1: -1]))
print('Amplitude mean: {}'.format(numpy.mean(new_amps)))
print('Phase mean: {}'.format(numpy.mean(new_phases)))

In [ ]:
# Optimal residuals for optimal redundant step
opt_w_alpha = degVis(ant_sep, res_rel_vis, *new_deg_params[[0, 2, 3]])
pred_opt_vis = gVis(opt_w_alpha, cRedG, new_gains)
opt_residuals =  obs_vis - pred_opt_vis
cplot(opt_residuals, xlabel='Baseline', ylabel='Residual')

In [ ]:
# Normalized amplitude residuals
plt.figure(figsize=(12, 8))
plt.plot(norm_residuals(numpy.abs(obs_vis), numpy.abs(pred_opt_vis)))
plt.xlabel('Baseline')
plt.ylabel('Normalized residual')
plt.ylim((-1, 1))
plt.show()

In [ ]:
# Phase residuals
diff_phases = numpy.angle(obs_vis) - numpy.angle(pred_opt_vis)
# wrap between {-pi, pi}
diff_phases_wrapped = (diff_phases + numpy.pi) % (2 * numpy.pi) - numpy.pi
plt.figure(figsize=(12,8))
plt.plot(diff_phases)
plt.plot(diff_phases_wrapped, label='wrapped')
plt.xlabel('Baseline')
plt.ylabel('Normalized residual')
plt.legend()
plt.show()

In [ ]:
# Residuals normalized by amplitude
norm_opt_residuals = norm_residuals(obs_vis, pred_opt_vis)
cplot(norm_opt_residuals, xlabel='Baseline', ylabel='Residual', ylim=(-1, 1))
print('Median absolute normalized residual - Real: {}, Imag: {}'\
      .format(*abs_residuals(norm_opt_residuals)))

# Comparing relative calibrations

To check the stability of the true visibilities for each baseline set, we perform relative calibration on neighbouring datasets in time, frequency, or JD, and compare their visibility solutions. These should be consistent up to the degenerate parameters $A$, $\Delta_x$ and $\Delta_y$. Marginalizing for these parameters with an MLE framework enables us to compare these solutions without having to resort to optimal calibration, which can be computationally expensive.

To compare datasets, we need to minimize:

- Gaussian distribution:

$$ \chi^2_{\text{deg}} (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left| U_{\alpha}' (\nu) - W_{\alpha} (\nu) \right|^2 }{\sigma_{\alpha}^2(\nu)} $$

- Cauchy distribution:

$$ -\ln(\mathcal{L}) (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha} (\nu)) + \ln \left( 1 + \left( \frac{\left| U_{\alpha}' (\nu) - W_{\alpha} (\nu) \right|}{\gamma_{\alpha} (\nu)} \right)^2 \right) $$

In [ ]:
JD2 = match_lst(JD, 2458099, tint=time_integration) # finding the JD_time of the zen_file
# that matches the LAST of the dataset used in 1.

zen_fn2 = find_zen_file(JD2)
bad_ants2 = get_bad_ants(zen_fn2)
flags_fn2 = find_flag_file(JD2, 'first') # import flags from firstcal

In [ ]:
hdraw2, RedG2, cMData2 = group_data(zen_fn2, pol, freq_channel, None, bad_ants2, flags_fn2)
cData2 = cMData2.filled()
flags2 = cMData2.mask

if cData2.shape[0] > 1:
    cData2 = cData2[0, ...]
    flags2 = flags2[0, ...]
cData2 = numpy.squeeze(cData2)
flags2 = numpy.squeeze(flags2)  

ants2 = numpy.unique(RedG2[:, 1:])
no_ants2 = ants2.size
no_unq_bls2 = numpy.unique(RedG2[:, 0]).size
cRedG2 = relabelAnts(RedG2)

print('Do the visibilities for JDs {} and {} have:\n'\
      'the same bad antennas? {}\n'\
      'the same flags? {}\n'\
      'the same redundant grouping? {}'.format(JD, JD2, (bad_ants == bad_ants2).all(), \
      (flags == flags2).all(), (RedG==RedG2).all()))

In [ ]:
# Find time integration in dataset 2 that corresponds to closest LST to that of dataset 1
# This currently assumes that dataset 2 contains the correct time integration...
time_integration2 = find_nearest(hdraw2.lsts, hdraw.lsts[time_integration])[1]

In [ ]:
plot_red_vis(cData2, RedG2, vis_type='amp')

In [ ]:
# Relative calibration for the 2nd dataset
obs_vis2 = cData2[time_integration2, :]
if not RP_results:
    res_rel2 = doRelCal(cRedG2, obs_vis2, no_unq_bls2, no_ants2, distribution=distribution, \
                        coords=rel_cal_coords, bounded=bounded_rel_cal, norm_gains=True)
else:
    res_rel2, _ = doRelCalRP(cRedG2, obs_vis2, no_unq_bls2, no_ants2, distribution=distribution, \
                             constr_phase=True, amp_constr='prod', bounded=True)

res_relx2 = numpy.array(res_rel2['x'])
if (rel_cal_coords == 'polar' or RP_results) and rot_phase and \
   (res_relx2[-2*no_ants::2] < 0).any():
    print('Rotating gain and visibility phases to have positive amplitudes.')
    res_relx2 = rotate_phase(res_relx2, no_unq_bls, norm_gains=True)
    split_coords2 = 'polar'
else:
    split_coords2 = rel_cal_coords
res_rel_vis2, res_rel_gains2 = split_rel_results(res_relx2, no_unq_bls2, \
                                                 coords=split_coords2)

print('Gain - average amp: {}, product of amps: {}, average phase: {}'\
      .format(numpy.abs(res_rel_gains2).mean(), numpy.prod(numpy.abs(res_rel_gains2)), \
      numpy.mean(numpy.angle(res_rel_gains2))))

In [ ]:
print('The negative log-likelihoods of the 1st and 2nd relative redundant calibrations '\
      'are:\n{} and\n{}'.format(res_rel['fun'], res_rel2['fun']))

In [ ]:
# Checking if normalizing the solutions changes anything...
# res_rel_vis, res_rel_gains = split_rel_results(norm_rel_sols(res_relx, \
#                                  no_unq_bls, coords=split_coords2), no_unq_bls, \
#                                  coords=split_coords2)
# res_rel_vis2, res_rel_gains2 = split_rel_results(norm_rel_sols(res_relx2, \
#                                    no_unq_bls, coords=split_coords2), no_unq_bls, \
#                                    coords=split_coords2)

In [ ]:
# Gain amplitudes for 2nd relative calibration results
res_relx[-2*no_ants:][::2]

In [ ]:
# Normalized gain amplitudes for 2nd relative calibration results
numpy.abs(res_rel_gains)

In [ ]:
# Visibility amplitudes from the 1st relative calibration
numpy.abs(res_rel_vis)

In [ ]:
# Visibility amplitudes from the 2nd relative calibration
numpy.abs(res_rel_vis2)

In [ ]:
numpy.angle(res_rel_vis).sum()

In [ ]:
numpy.angle(res_rel_vis2).sum()

In [ ]:
# Translating between relatively calibrated visibility sets
res_deg = doDegVisVis(RedG, hdraw.antpos, res_rel_vis, res_rel_vis2, \
                      distribution=distribution)
deg_tr_params = res_deg['x']

print('Degenerate parameters from degenerate fitting are: {}'.format(deg_tr_params))
print('The negative log-likelihood for this fitting is: {}'.format(res_deg['fun']))

In [ ]:
# Residuals for degenerate comparison
deg_w_alpha = degVis(ant_sep, res_rel_vis, *deg_tr_params)
deg_residuals = res_rel_vis2 - deg_w_alpha
cplot(deg_residuals, xlabel='Redundant baseline type', ylabel='Residual')

In [ ]:
# Degenerate residuals normalized by amplitude
norm_deg_residuals = norm_residuals(res_rel_vis2, deg_w_alpha)
cplot(norm_deg_residuals, xlabel='Redundant baseline type', ylabel='Residual')
print('Median absolute normalized residual - Real: {}, Imag: {}'\
      .format(*abs_residuals(norm_deg_residuals)))

## Comparing optimally calibrated solutions

In [ ]:
ant_pos2 = flt_ant_pos(hdraw2.antpos, ants2)
ant_sep2 = red_ant_sep(RedG2, hdraw2.antpos)

res_opt2 = doOptCal(cRedG2, obs_vis2, no_ants2, ant_pos2, ant_sep2, res_rel_vis2, \
                    distribution=distribution, ref_ant_idx=ref_ant_idx)

In [ ]:
_, new_deg_params2 = numpy.split(res_opt2['x'], [no_ants*2,])
opt_w_alpha2 = degVis(ant_sep, res_rel_vis2, *new_deg_params2[[0, 2, 3]])

In [ ]:
res_opt_deg = doDegVisVis(RedG, hdraw.antpos, opt_w_alpha, opt_w_alpha2, \
                          distribution=distribution)
deg_opt_tr_params = res_deg['x']

print('Degenerate parameters from degenerate fitting are: {}'.format(deg_tr_params))
print('The negative log-likelihood for this fitting is: {}'.format(res_deg['fun']))

In [ ]:
# Degenerate residuals normalized by amplitude
deg_opt_w_alpha = degVis(ant_sep, opt_w_alpha, *deg_opt_tr_params)
norm_deg_opt_residuals = norm_residuals(opt_w_alpha2, deg_opt_w_alpha)
cplot(norm_deg_opt_residuals, xlabel='Redundant baseline type', ylabel='Residual')
print('Median absolute normalized residual - Real: {}, Imag: {}'\
      .format(*abs_residuals(norm_deg_opt_residuals)))